In [2]:
import pandas as pd

houses =pd.read_csv('version_D.csv')
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27300 entries, 0 to 27299
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       27300 non-null  int64  
 1   bathrooms      27300 non-null  float64
 2   area           27300 non-null  int64  
 3   lot_size       27300 non-null  float64
 4   floors         27300 non-null  int64  
 5   age            27300 non-null  int64  
 6   garage         27300 non-null  int64  
 7   fireplace      27300 non-null  int64  
 8   renovated      27300 non-null  int64  
 9   neighborhood   27300 non-null  int64  
 10  distance_city  27300 non-null  float64
 11  price          27300 non-null  float64
 12  price_hilo     27300 non-null  int64  
dtypes: float64(4), int64(9)
memory usage: 2.7 MB


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
y = houses.price
X = houses.drop(['price','price_hilo'], axis = 1)
y_binned = pd.qcut(y, q = 100)
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state=617, 
                                                    train_size=0.7, 
                                                    stratify=y_binned)